In [9]:
import numpy as np
import  math
from tabulate import tabulate
from random import random


# Datos proporcionados
productos = ["P1", "P2", "P3", "P4", "P5"]
ventas_promedio = [119, 140, 155, 158, 176]
volumenes_productos = [1, 0.8, 1.2, 1.5, 0.6]
numero_clientes = 10
productos_c_cliente = []

In [2]:
poisson = lambda x, t: (t**(x)*math.exp(-t))/math.factorial(x)
poissonAcum = lambda x,t, include: sum([poisson(i,t) for i in range(x + (1 if include else 0))])
tabla = {
"# Productos":     [i for i in range(11)],
"poisson"  :    [poisson(i, 2) for i in range(11)],
"Acumulado" : [poissonAcum(i, 2, True) for i in range(11)],
"limInf":   [0] + [poissonAcum(i, 2, True) for i in range(10)],
"limSup":   [poissonAcum(i, 2, True) for i in range(11)]
}

print("--- TABLA POSSION PARA RULEATA ---")
print(tabulate(tabla, headers="keys"))

--- TABLA POSSION PARA RULEATA ---
  # Productos      poisson    Acumulado    limInf    limSup
-------------  -----------  -----------  --------  --------
            0  0.135335        0.135335  0         0.135335
            1  0.270671        0.406006  0.135335  0.406006
            2  0.270671        0.676676  0.406006  0.676676
            3  0.180447        0.857123  0.676676  0.857123
            4  0.0902235       0.947347  0.857123  0.947347
            5  0.0360894       0.983436  0.947347  0.983436
            6  0.0120298       0.995466  0.983436  0.995466
            7  0.00343709      0.998903  0.995466  0.998903
            8  0.000859272     0.999763  0.998903  0.999763
            9  0.000190949     0.999954  0.999763  0.999954
           10  3.81899e-05     0.999992  0.999954  0.999992


In [27]:
def ruleata(frequencia):
    r = random()
    limites = (np.array(ventas_promedio)/sum(ventas_promedio)).cumsum()
    for i in range(len(limites)):
        if r<limites[i]:
            return i
rangos = list(zip(tabla["limInf"], tabla["limSup"], tabla["# Productos"]))
numPiezas = lambda x: [rango[2] for rango in rangos if rango[0]<x<=rango[1] or (rango[1] == rangos[10][1] and x>rango[1]) ][0]
num = np.vectorize(numPiezas)
unidadesPorCliente = num(np.random.random_sample(numero_clientes,))
unidadesPorCliente

array([5, 4, 5, 1, 3, 3, 5, 3, 2, 3])

In [28]:
# Simulación de la venta de la empresa para un día
productos_c_cliente = []
for cliente in range(numero_clientes):

    # Selección aleatoria de productos
    productosPorCliente= []
    for i in range(unidadesPorCliente[cliente]):
        productosPorCliente.append(productos[ruleata(ventas_promedio)])
    productos_c_cliente.append(productosPorCliente.copy())

In [29]:
productos_c_cliente

[['P5', 'P5', 'P3', 'P5', 'P3'],
 ['P5', 'P1', 'P2', 'P1'],
 ['P3', 'P3', 'P1', 'P2', 'P5'],
 ['P1'],
 ['P3', 'P4', 'P3'],
 ['P4', 'P5', 'P2'],
 ['P2', 'P5', 'P3', 'P5', 'P1'],
 ['P5', 'P3', 'P3'],
 ['P5', 'P5'],
 ['P2', 'P1', 'P5']]